In [17]:
from typing_extensions import Self
from enum import Enum

In [18]:
# Load the input
with open("day3_input.txt") as f:
    day3_input = f.read()

In [19]:
class BlockType(Enum):
    NUMERIC = 1
    SYMBOLIC = 2
    BLANK = 3

    @classmethod
    def GetForChar(cls: Self, char: str):
        assert(len(char) == 1)
        if char.isnumeric(): return cls.NUMERIC
        # Check order is important here, need to exclude full stops and asterisks before other chars
        if char == ".": return cls.BLANK
        # Check probably redundant, not expecting any space inputs
        if not char.isspace(): return cls.SYMBOLIC


# Block class
class Block:
    # Contents
    contents: str

    # Type - 
    block_type: BlockType

    # Position information
    line_number: int
    start_index: int
    end_index: int

    # Links to other blocks
    links: set[Self]

    def __init__(self: Self, block_type: BlockType, contents: str, line_number: int, start_index: int, end_index: int):
        self.block_type = block_type
        self.contents = contents
        self.line_number = line_number
        self.start_index = start_index
        self.end_index = end_index
        self.links = set()

    # Just helpful to be able to print
    def __str__(self: Self) -> str:
        return f"Block on line {self.line_number} starting at index {self.start_index} with contents: '{self.contents}'"
    def __repr__(self) -> str:
        return str(self)
    # Need these to do dictionary / set operations and comparisons
    def __hash__(self: Self) -> int:
        return hash(hash(self.line_number) + hash(self.start_index) + hash(self.contents))
    def __eq__(self: Self, other: Self) -> bool:
        return self.line_number == other.line_number and self.start_index == other.start_index and self.contents == other.contents
    
    # Link two blocsk
    @classmethod
    def LinkBlocks(cls: Self, block_a: Self, block_b: Self):
        block_a.links.add(block_b)
        block_b.links.add(block_a)

    def ResetLinks(self: Self):
        self.links.clear()
    
    # Utilities to check if blocks are adjacent to one another
    def containsindex(self: Self, index: int) -> bool:
        return self.start_index <= index and self.end_index >= index

    def isadjacent(self: Self, other: Self) -> bool:
        # Reject identical blocks
        if self == other: return False
        # Quick check for line numbers
        if abs(self.line_number - other.line_number) > 1: return False

        # Check the other block includes to see if they include the index one below our start index
        # or one above our end index. If they do they are either adjacent on the same line or
        # diagonally adjacent on lines above / below
        if other.containsindex(self.start_index-1) or other.containsindex(self.end_index+1): return True

        # Only the line above or line below need to check the same indices. We're assuming here that
        # the blocks we're given are a valid mapping i.e. you wouldn't get two blocks covering the
        # same index of the same line. This assumption saves a bit of time.
        if self.line_number == other.line_number: return False

        # For blocks on the line above and below we have to check every index but we can stop as soon
        # as a shared one is found
        for idx in range(self.start_index,self.end_index + 1):
            if other.containsindex(idx): return True

        # Didn't find any early exits or connections, must not be adjacent
        return False

    # Check if this block represents a valid part number according to the part 1 criteria
    # i.e. a numeric block that is connected to at least one symbolic block
    def ispartnumber(self: Self) -> bool:
        return self.block_type == BlockType.NUMERIC and any(linked_block.block_type == BlockType.SYMBOLIC for linked_block in self.links)
    
    # Check if this block represents a gear according to the part 2 criteria
    # i.e. a symbolic block containing "*" connected to exactly 2 numeric blocs
    def isgear(self: Self) -> bool:
        return self.block_type == BlockType.SYMBOLIC and self.contents == "*" and len([block for block in self.links if block.block_type == BlockType.NUMERIC]) == 2

    # Gear ratio is just the product of the connected numeric blocks
    def gearratio(self: Self) -> int:
        assert(self.isgear())
        ratio = 1
        for block in self.links:
            if block.block_type == BlockType.NUMERIC: ratio = ratio*int(block.contents)
        return ratio

In [20]:
# Turn a line of input into blocks
def ParseBlocksFromInputLine(line: str, line_number: int) -> list[Block]:
    assert(len(line) > 0)
    blocks = []

    # i tracks the start index of the current block, j tracks the end index
    # Note that we'll slice the string with [i:j+1] so that when i and j are identical we still
    # get a character
    i = 0
    j = 0

    # Current block type
    block_type = BlockType.GetForChar(line[0])

    # Keep going until j is past the end of the string
    while j < len(line) - 1:
        # Look at the next char along, and compare its type to the current block
        next_char = line[j+1]
        next_char_type = BlockType.GetForChar(next_char)
        if next_char_type == block_type:
            # Same type, just increment j
            j = j + 1
        else:
            # Different type, create the current block
            block = Block(block_type=block_type, contents=line[i:j+1], line_number=line_number, start_index=i, end_index=j)
            # print(f"Created block: {block}")
            blocks.append(block)
            # Move i and j to the next char
            i = j + 1
            j = i
            # Update block type for future checks
            block_type = next_char_type
    
    # Create the final block
    block = Block(block_type=block_type, contents=line[i:j+1], line_number=line_number, start_index=i, end_index=j)
    blocks.append(block)
    return blocks

In [21]:
# Parse the input into blocks. We'll store these per-line, every line should have at least one
# block so we can just index this array the same as the lines themselves i.e. every block in
# line_blocks[n] will have line_number = n
blocks_by_line = []
line_number = 0
for line in day3_input.split("\n"):
    blocks_by_line.append(ParseBlocksFromInputLine(line=line, line_number=line_number))
    line_number = line_number + 1

In [22]:
# Build links between blocks
[[block.ResetLinks() for block in line_blocks] for line_blocks in blocks_by_line]
for line_number, line_blocks in enumerate(blocks_by_line):
    for block in line_blocks:
        # Link to blocks on the line above if this isn't the first line
        # This should also give us the linking to lines above because the next line will check this
        # one and we make mutual links
        if line_number > 0:
            for block_above in blocks_by_line[line_number-1]:
                if block.isadjacent(block_above):
                    Block.LinkBlocks(block,block_above)
        # Link to blocks on the same line, but ignore the current block
        for other_block in line_blocks:
            if other_block != block and block.isadjacent(other_block): Block.LinkBlocks(block,other_block)

In [ ]:
# Work out the answer to part 1
print(f"Answer (part one): {sum([sum([int(block.contents) for block in line_blocks if block.ispartnumber()]) for line_blocks in blocks_by_line])}")

In [ ]:
# Work out the answer to part 2
print(f"Answer (part two): {sum([sum([block.gearratio() for block in line_blocks if block.isgear()]) for line_blocks in blocks_by_line])}")